# Project 3: Poisonous mushrooms

In this project, you'll investigate properties of mushrooms. This classic dataset contains over 8000 observations, where each mushroom is described by a variety of features like color, odor, etc., and the target variable is an indicator for whether the mushroom is poisonous. Since all the observations are categorical, I've binarized the feature space. Look at the feature_names below to see all 126 binary names.

You'll start by running PCA to reduce the dimensionality from 126 down to 2 so that you can easily visualize the data. In general, PCA is very useful for visualization (though sklearn.manifold.tsne is known to produce better visualizations). Recall that PCA is a linear transformation. The 1st projected dimension is the linear combination of all 126 original features that captures as much of the variance in the data as possible. The 2nd projected dimension is the linear combination of all 126 original features that captures as much of the remaining variance as possible. The idea of dense low dimensional representations is crucial to machine learning!

Once you've projected the data to 2 dimensions, you'll experiment with clustering using KMeans and density estimation with Gaussian Mixture Models. Finally, you'll train a classifier by fitting a GMM for the positive class and a GMM for the negative class, and perform inference by comparing the probabilities output by each model.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from matplotlib.colors import LogNorm

Load feature names.

In [ ]:
feature_names = []
with open('../Data/mushroom.map') as fmap:
    for line in fmap:
        [index, name, junk] = line.split()
        feature_names.append(name)

print('Loaded feature names:', len(feature_names))

Load data. The data is sparse in the input file, but there aren't too many features, so we'll use a dense representation, which is supported by all sklearn objects.

In [ ]:
X, Y = [], []

with open('../Data/mushroom.data') as fdata:
    for line in fdata:
        items = line.split()
        Y.append(int(items.pop(0)))
        x = np.zeros(len(feature_names))
        for item in items:
            feature = int(item.split(':')[0])
            x[feature] = 1
        X.append(x)

# Convert these lists to numpy arrays.
X = np.array(X)
Y = np.array(Y)

# Split into train and test data.
train_data, train_labels = X[:7000], Y[:7000]
test_data, test_labels = X[7000:], Y[7000:]

# Check that the shapes look right.
print(train_data.shape, test_data.shape)

(1) Run Principal Components Analysis on the data. Show what fraction of the total variance in the training data is explained by the first k principal components, for k in [1, 50].

In [ ]:
#def P1():
### STUDENT START ###

for k in range(1, 50):
    pca = PCA(n_components=k)
    pca.fit(train_data)
    print("Explained variance with {} components: {:.3f}".format(k, np.sum(pca.explained_variance_ratio_)))

### STUDENT END ###

#P1()

(2) PCA can be very useful for visualizing data. Project the training data down to 2 dimensions and plot it. Show the positive (poisonous) cases in blue and the negative (non-poisonous) in red. Here's a reference for plotting: http://matplotlib.org/users/pyplot_tutorial.html

In [ ]:
#def P2():
### STUDENT START ###

pca = PCA(n_components=2)

# get PCA transformed data
new_data = pca.fit_transform(train_data)
new_x = [x[0] for x in new_data]
new_y = [x[1] for x in new_data]

poison_x = [x for i, x in enumerate(new_x) if train_labels[i]==1]
poison_y = [y for i, y in enumerate(new_y) if train_labels[i]==1]

non_poison_x = [x for i, x in enumerate(new_x) if train_labels[i]!=1]
non_poison_y = [y for i, y in enumerate(new_y) if train_labels[i]!=1]

plt.figure(figsize=(10,10))
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.title("Mushrooms plotted by 2 principle components")
plt.plot(poison_x, poison_y, 'b.')
plt.plot(non_poison_x, non_poison_y, 'r.')
plt.gca().legend(("Poison", "Non-Poison"), loc=2)
plt.show()
### STUDENT END ###

#P2()

(3) Run KMeans with [1,16] clusters over the 2d projected data. Mark each centroid cluster and plot a circle that goes through the most distant point assigned to each cluster.

In [ ]:
for k in range(1,16):
    kmeans = KMeans(n_clusters = k)
    cluster_labels = kmeans.fit_predict(new_data)
    cluster_distances = kmeans.transform(new_data)

    cluster_dicts_list = []
    
    for i in range(k):
        cluster_dict = {}
        # get x and y for each cluster
        cluster_dict['x'] = [x for e, x in enumerate(new_x) if cluster_labels[e]==i]
        cluster_dict['y'] = [y for e, y in enumerate(new_y) if cluster_labels[e]==i]
        cluster_dict['center'] = kmeans.cluster_centers_[i] 
        
        # find centroid, furthest point, and radius
        # for each cluster
        this_cluster_distances = [d[i] for e,d in enumerate(cluster_distances) if cluster_labels[e] == i]
        max_distance_point = this_cluster_distances.index(max(this_cluster_distances))
        
        cluster_dict['radius'] = np.sqrt((cluster_dict['x'][max_distance_point] - cluster_dict['center'][0])**2
                                         + (cluster_dict['y'][max_distance_point] - cluster_dict['center'][1])**2)
    
        cluster_dicts_list.append(cluster_dict)
    
    fig, ax = plt.subplots(figsize=(10,10))
    ax.axis("equal")
    for cluster_dict in cluster_dicts_list:
        plt.title("Clusters with k={}".format(k))
        plt.plot(cluster_dict['x'], cluster_dict['y'], '.')
        plt.plot(cluster_dict['center'][0], cluster_dict['center'][1], 'r.')
        circle = plt.Circle((cluster_dict['center'][0], 
                             cluster_dict['center'][1]), cluster_dict['radius'], fill=False)
        ax.add_artist(circle)
    plt.show()

(4) Fit a Gaussian Mixture Model for the positive examples in your 2d projected data. Plot the estimated density contours as shown here: http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_pdf.html#example-mixture-plot-gmm-pdf-py. Vary the number of mixture components from 1-4 and the covariance matrix type ('spherical', 'diag', 'tied', 'full').

In [ ]:
#def P4():
### STUDENT START ###
## TODO: check with others - this works, but doesn't quite look right
poison_sample = [[poison_x[i], poison_y[i]] for i in range(len(poison_x))]
poison_sample = np.asarray(poison_sample)

component_options = list(range(1,4))
cov_matrix_options = ['spherical', 'diag', 'tied', 'full']

for component in component_options:
    for cov_matrix in cov_matrix_options:
        gm = GaussianMixture(n_components = component,
                             covariance_type = cov_matrix)
        gm.fit(poison_sample)
        
        x = np.linspace(-10., 15.)
        y = np.linspace(-10., 20.)
        X, Y = np.meshgrid(x, y)
        XX = np.array([X.ravel(), Y.ravel()]).T
        Z = -gm.score_samples(XX)
        Z = Z.reshape(X.shape)

        CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0),
                         levels=np.logspace(0, 3, 10))
        CB = plt.colorbar(CS, shrink=0.8, extend='both')
        plt.scatter(poison_sample[:, 0], poison_sample[:, 1], .8)

        plt.title('Estimated density contours: {} components, {} cov matrix'.format(component, 
                                                                                    cov_matrix))
        plt.axis('tight')
        plt.show()
### STUDENT END ###

#P4()

(5) Fit two 4-component full covariance GMMs, one for the positive examples and one for the negative examples in your 2d projected data. Predict the test examples by choosing the label for which the model gives a larger probability (use GMM.score). What is the accuracy?

In [ ]:
#def P5():
### STUDENT START ###

# make the two gmms
pos_gm = GaussianMixture(n_components = 4,
                         covariance_type = 'full')
pos_gm.fit(poison_sample)

non_poison_sample = [[non_poison_x[i], non_poison_y[i]] for i in range(len(non_poison_x))]
non_poison_sample = np.asarray(non_poison_sample)

non_pos_gm = GaussianMixture(n_components = 4,
                         covariance_type = 'full')
non_pos_gm.fit(non_poison_sample)

# transform test data with PCA

test_pca_data = pca.transform(test_data)

# classify examples using the two gmms 

pred_labels = []

for example in test_pca_data:
    poison_score = pos_gm.score(example.reshape(1,-1))
    non_poison_score = non_pos_gm.score(example.reshape(1,-1))
    if np.exp(poison_score) > np.exp(non_poison_score):
        pred_labels.append(1)
    else:
        pred_labels.append(0)
        
# see how many it got right

truthy_array = pred_labels == test_labels
print("Accuracy: {:.3f}".format(sum(truthy_array) / len(truthy_array)))

### STUDENT END ###

#P5()

(6) Ideally, we'd like a model that gives the best accuracy with the fewest parameters. Run a series of experiments to find the model that gives the best accuracy with no more than 50 parameters. For example, with 3 PCA components and 2-component diagonal covariance GMMs, you'd have:

( (3 mean vector + 3 covariance matrix) x 2 components ) x 2 classes = 24 parameters

You should vary the number of PCA components, the number of GMM components, and the covariance type.

In [ ]:
#def P6():
### STUDENT START ###

## todo: still need to understand the other types of covariance well enough to do calculation
# i think this calculation only works for diagonal covariance 

## fit PCA on train
def fit_pca_return_two_groups(num_components):
    pca = PCA(n_components = num_components)
    new_data = pca.fit_transform(train_data)
        
    poison_data = [[entry] for i, entry in enumerate(new_data) if train_labels[i]==1]
    poison_data = np.concatenate(poison_data, axis=0)
    
    non_poison_data = [[entry] for i, entry in enumerate(new_data) if train_labels[i]==0]
    non_poison_data = np.concatenate(non_poison_data, axis=0)
    
    new_test_data = pca.transform(test_data)
    return poison_data, non_poison_data, new_test_data

## train the two gmms
def train_both_gmms(num_components,
                    cov_type,
                    poison_data,
                    non_poison_data):
    pos_gm = GaussianMixture(n_components = num_components,
                             covariance_type = cov_type)
    pos_gm.fit(poison_data)

    non_pos_gm = GaussianMixture(n_components = num_components,
                                 covariance_type = cov_type)
    non_pos_gm.fit(non_poison_data)
    
    return pos_gm, non_pos_gm

def predict_on_test_data(test_data,
                         pos_gm,
                         non_pos_gm):
    pred_labels = []

    for example in test_data:
        poison_score = pos_gm.score(example.reshape(1,-1))
        non_poison_score = non_pos_gm.score(example.reshape(1,-1))
        if np.exp(poison_score) > np.exp(non_poison_score):
            pred_labels.append(1)
        else:
            pred_labels.append(0)
    
    truthy_array = pred_labels == test_labels
    accuracy = sum(truthy_array) / len(truthy_array)
    print("Accuracy: {:.3f}".format(sum(truthy_array) / len(truthy_array)))
    return accuracy

    
## predict on test
## calculate predictions

pca_components = list(range(1,5))
gmm_components = list(range(1,5))

highest_so_far = 0
for pca_c in pca_components:
    for gmm_c in gmm_components:
        if ((pca_c + pca_c) * gmm_c) * 2 <= 50:
            print("\nNum PCA components: {}".format(pca_c))
            print("Num GMM components: {}".format(gmm_c))
            print("Num parameters: {}".format(((pca_c + pca_c) * gmm_c) * 2))
            p_d, n_p_d, t_d = fit_pca_return_two_groups(pca_c)
            pos_gm, non_pos_gm = train_both_gmms(gmm_c,
                                                 'diag',
                                                 p_d,
                                                 n_p_d)
            acc = predict_on_test_data(t_d,
                                       pos_gm,
                                       non_pos_gm)
            if acc > highest_so_far:
                print("Highest so far!")
                highest_so_far = acc

### STUDENT END ###

#P6()

In [ ]:
### OLD - this is when i was not trying to do it dynamically
#def P3():
### STUDENT START ###
kmeans = KMeans(n_clusters = 3)
cluster_labels = kmeans.fit_predict(new_data)
cluster_centers_x = [x[0] for x in kmeans.cluster_centers_]
cluster_centers_y = [x[1] for x in kmeans.cluster_centers_]

cluster_1_x = [x for i, x in enumerate(new_x) if cluster_labels[i]==0]
cluster_1_y = [y for i, y in enumerate(new_y) if cluster_labels[i]==0]

cluster_2_x = [x for i, x in enumerate(new_x) if cluster_labels[i]==1]
cluster_2_y = [y for i, y in enumerate(new_y) if cluster_labels[i]==1]

cluster_3_x = [x for i, x in enumerate(new_x) if cluster_labels[i]==2]
cluster_3_y = [y for i, y in enumerate(new_y) if cluster_labels[i]==2]

# finding furthest point
cluster_distances = kmeans.transform(new_data)
cluster_1_distances = [d[0] for i,d in enumerate(cluster_distances) if cluster_labels[i] == 0]
max_distance_point = cluster_1_distances.index(max(cluster_1_distances))
radius = np.sqrt((cluster_1_x[max_distance_point] - cluster_centers_x[0])**2
                 + (cluster_1_y[max_distance_point] - cluster_centers_y[0])**2)
print(radius)

## find euclidean distance from centroid to max distance point
# that should be radius of circle 

fig, ax = plt.subplots(figsize=(10,10))
ax.axis("equal")
plt.plot(cluster_1_x, cluster_1_y, 'y.')
plt.plot(cluster_2_x, cluster_2_y, 'g.')
plt.plot(cluster_3_x, cluster_3_y, 'b.')
plt.plot(cluster_centers_x, cluster_centers_y, 'b.')
circle = plt.Circle((cluster_centers_x[0], cluster_centers_y[0]), radius, fill=False)
ax.add_artist(circle)
plt.plot(cluster_1_x[max_distance_point], cluster_1_y[max_distance_point], 'r.')

### STUDENT END ###

#P3()